# Instruções para reproduzir os resultados do modelo final

In [ ]:
# instalar todas as bibliotecas utilizadas
!pip install -r requirements.txt

## 1. Pré-processar o conjunto de dados

Execute o notebook jupyter 'pre_precessing_pipeline.ipynb' que está no diretório 'processing'. Você deve obter como resultado o arquivo 'esic2023_cleaned.csv' armazenado em 'data'

## 2. Vetorizar conjunto de dados limpo

Execute o notebook jupyter 'vectorizer.ipynb' que está no diretório 'processing'. Você deve obter como resultado os arquivos {'X_train_Tfidf.npz', 'X_test_Tfidf.npz', 'y_train.npy', 'y_test.npy'} armazenado em 'vects'

## 3. Treinar o modelo de SVM

Execute as células abaixo para, por fim, treinar o modelo final

In [11]:
import numpy as np
from scipy.sparse import load_npz

from sklearn import svm

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import warnings

warnings.filterwarnings("ignore")

### Carregar vetores de treino e teste

In [12]:
X_train_Tfidf = load_npz('vects/X_train_Tfidf.npz')
X_test_Tfidf = load_npz('vects/X_test_Tfidf.npz')

y_train = np.load('vects/y_train.npy', allow_pickle=True)
y_test = np.load('vects/y_test.npy', allow_pickle=True)

### Otimização de hiperparametros

In [13]:
SVM = svm.SVC()

In [14]:
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [1.0, 2.0, 3.0]
}

In [15]:
grid_search = GridSearchCV(SVM, param_grid, cv=10, scoring='accuracy')

grid_search.fit(X_train_Tfidf, y_train)

print("Melhores hiperparâmetros:", grid_search.best_params_)

Melhores hiperparâmetros: {'C': 2.0, 'kernel': 'rbf'}


In [16]:
accuracy = grid_search.score(X_test_Tfidf, y_test)
print("Acurácia no conjunto de teste:", accuracy)

Acurácia no conjunto de teste: 0.5816666666666667


### Avaliação

In [18]:
SVM = svm.SVC(**grid_search.best_params_)

In [21]:
scores = cross_val_score(SVM, X_train_Tfidf, y_train, cv=10).mean()

print("{:.2f}".format(scores*100), '%')

56.55 %


### Teste

In [22]:
SVM.fit(X_train_Tfidf, y_train)

y_pred = SVM.predict(X_test_Tfidf)

scores = accuracy_score(y_pred=y_pred, y_true=y_test)

print("{:.2f}".format(scores*100), '%')

58.17 %
